### Joey's version in root CRA folder

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Joey's-version-in-root-CRA-folder" data-toc-modified-id="Joey's-version-in-root-CRA-folder-0.0.1"><span class="toc-item-num">0.0.1&nbsp;&nbsp;</span>Joey's version in root CRA folder</a></span></li></ul></li></ul></li><li><span><a href="#Transform-text-files-to-emails" data-toc-modified-id="Transform-text-files-to-emails-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Transform text files to emails</a></span></li><li><span><a href="#Make-dicts-and-def-functions" data-toc-modified-id="Make-dicts-and-def-functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Make dicts and def functions</a></span></li><li><span><a href="#Usage" data-toc-modified-id="Usage-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Usage</a></span></li><li><span><a href="#non-function-code-below" data-toc-modified-id="non-function-code-below-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>non function code below</a></span></li></ul></div>

# Transform text files to emails
* File names contain: 
    * an immutable email id
    * the file name that contains the email
    * the line number that locates the email

# Make dicts and def functions

**All cells below are necessary**

In [102]:
%load_ext autoreload
%autoreload 2

In [103]:
import os
import io
import sys
import re
import csv
import pandas as pd
from tqdm.notebook import tqdm, tnrange
from datetime import datetime
from collections import defaultdict
from dataclasses import dataclass, field 
import function_library3 as fct_lib

In [2]:
# regex in headerReLib must have one and ONLY one pair of parenthese.
headerReLib = {}  # dict{regex: str}
headerReLib[re.compile(r'^From\:\s?(.*)')] = 'From'
headerReLib[re.compile(r'^Sent\:\s?(.*)')] = 'Sent'
headerReLib[re.compile(r'^Date\:\s?(.*)')] = 'Sent'
headerReLib[re.compile(r'^To\:\s?(.*)')] = 'To'
headerReLib[re.compile(r'^C[Cc]\:\s?(.*)')] = 'CC'
headerReLib[re.compile(r'^B[Cc][Cc]\:\s?(.*)')] = 'BCC'
headerReLib[re.compile(r'^Subject\:\s?(.*)')] = 'Subject' 
headerReLib[re.compile(r'^Attachments\:\s?(.*)')] = 'Attachments'  # Attachment: is not reliable in 9-1
headerReLib[re.compile(r'Importance\:\s?(.*)')] = 'Importance'
headerReLib[re.compile(r'Priority\:\s?(.*)')] = 'Importance'



In [3]:
cleanReLib = {} # dict{regex: str}
cleanReLib[re.compile(r'^Page\s\d+$')] = 'Page number'
cleanReLib[re.compile(r'^Page\s\d+\sof\s\d+$')] = 'Page number'
cleanReLib[re.compile(r'^\d{,3}\-\d{,3}$')] = 'Page number and file number'


In [4]:
typoLib = {} # dict{str: str}
typoLib['ARachments'] = 'Attachments'
typoLib['AFachments'] = 'Attachments'


In [70]:
def saveEmail(email, sv_arg):
    # save in specific order
    saving_order = ['From', 'Sent', 'To', 'CC', 'Bcc', 'Subject', 'Attachments', 'Importance', 'Body', 'isThread', 'isAutoMessage', 'isDisplacement', 'hasAllCapLine', 'hasBadDate']
    bool_headers = ['isThread', 'isAutoMessage', 'hasAllCapLine', 'hasBadDate', 'isDisplacement']
    
    str_email_count = f"%05d" % sv_arg.email_count
    filenm = str(str_email_count)+'_fn_'+sv_arg.infile.split('.')[0]+'_ln_'+str(sv_arg.line_no)+'.txt'
    filenm_path = sv_arg.outputpath + '/' + filenm # will not work on Windows
    
    with open(filenm_path, 'w+', encoding = 'utf-8') as f_out:
    #with open(sv_arg.outputpath + '/'+str(sv_arg.email_count)+'-fn_'+sv_arg.infile.split('.')[0]+'-ln_'+str(sv_arg.line_no)+'.txt', 'w+', encoding = 'utf-8') as f_out:
        for key in saving_order:
            line_no = sv_arg.line_nb[key]
            print(f"line_no: {line_no}, key: {key}")
            char_pos_eol = [sv_arg.char_pos_eol[lnb] for lnb in line_no]
            if key in email:
                if key == 'Body':
                    f_out.write(key+': '+'\n'.join(email[key]) + '\n') # for body, join by space '\n'
                elif key in bool_headers:
                    f_out.write(f'{line_no}, {char_pos_eol} \n' + key+': '+ str(email[key]) +'\n') # for bool type
                else:
                    f_out.write(f'{line_no}, {char_pos_eol} \n' + key+': '+' '.join(email[key]) + '\n')  # for others, join by ' '
    sv_arg.email_count += 1

In [49]:
def validEmail(email, sv_arg, save=True, saveAny=False):
    valid = False
    # email is valid if 2 or more sections are found
    checking_section = ["From", "To", "Sent"]
    valid_count = 0
    for section in checking_section:
        if section in email:
            valid_count += 1
    if valid_count >= 2:
        valid = True
    if saveAny or (save and valid):
        saveEmail(email, sv_arg)
        print(f"GE: Email saved, line_no: {sv_arg.line_no}, valid_count: {valid_count}")
    else:
        print(
            f"GE: Email not saved, line_no: {sv_arg.line_no}, valid_count: {valid_count}"
        )
    return valid

In [50]:
def completeEmail(email, sv_arg, spt_arg):
    # all lines in stack to body. even empty
    email["Body"] = email["Body"] + spt_arg.stack
    sv_arg.line_nb["Body"].append(sv_arg.line_no)
    validEmail(email, sv_arg, save=True, saveAny=False)
    
    print(f"GE: Reset stack, displacement_sections, prev_section ({spt_arg.prev_section})")

    spt_arg.stack = []
    spt_arg.displacement_sections = []
    spt_arg.prev_section = None  # GE: Correction. 2022-01-20

In [51]:
def isUselessLine(line, file_no = ''):
    if line == '':
        return True
    if line == file_no:
        return True
    for regex, cleaning_reason in cleanReLib.items(): 
        if regex.match(line):
            return True
    return False

In [52]:
def getFileNumber(infile):
    spt = infile.split('-')
    file_no = spt[0]
    if len(spt)>1 and spt[1].isdigit():
        file_no = file_no + '-' + spt[1]
    return file_no

In [53]:
@dataclass
class SavingArgs:
    inputpath: str # path to a folder ending with /
    outputpath: str # path to a folder ending with /
    infile: str = '' # filename only
    email_count: int = 0
    line_no: int = -1
    
@dataclass
class SplittingArgs:
    prev_section: str = None
    stack: list = field(default_factory=list)
    displacement_sections: list = field(default_factory=list)
    capTolerance: int = 0
#     appendingMode: bool = False   # Autoreply and attachment must be treated differently
    isAutoMessage: bool = False
    isDisplacement: bool = False
    isAttachment: bool = False

In [106]:
def isStringDate(string):
    return False

filters = fct_lib.create_date_filters()

In [132]:
def print_email_linenb(msg, email, line_nb):
    print(f"*** email, line_nb: {msg} ***")
    for k, v in email.items():
        print(f"header: {k}: {line_nb[k]}")
        print(f"        {v}")

In [189]:
def process(sv_arg):
    # define the bad date regex
    re_wrong_date = re.compile(r"\d{1,2}:\d{2}")

    f_in = open(sv_arg.inputpath + sv_arg.infile, encoding="utf8")
    file_no = getFileNumber(sv_arg.infile)

    # initialize SplittingArgs and email object
    spt_arg = SplittingArgs()
    email = defaultdict(list)
    sv_arg.line_nb = defaultdict(list)
    sv_arg.char_pos_eol = defaultdict(int)

    char_pos = 0  # character position within the text
    for line_no, line in enumerate(f_in.readlines()):
        sv_arg.line_no = line_no
        if line_no > 233:
            print("===> line_no: ", line_no)
        if line_no == 0:
            sv_arg.char_pos_eol[line_no] = (
                len(line) - 1
            )  #   Subtract 1 for use in  vi (shift nb chracters taking \n into account)
        else:
            sv_arg.char_pos_eol[line_no] = (
                sv_arg.char_pos_eol[line_no - 1] + len(line) - 1
            )
        # print(f"Char pos: {sv_arg.char_pos_eol[line_no]}")
        # decrement capTolerance (initial value is 2)
        spt_arg.capTolerance -= 1

        # if line start with >, it's in a thread
        line = line.strip("\f").strip()
        if len(line) > 0 and line[0] == ">":
            email["isThread"] = True
            sv_arg.line_nb["isThread"].append(line_no)
        line = line.strip(">").strip()

        # Skip some useless rows, including empty rows
        if isUselessLine(line, file_no):
            continue

        # correct typos
        for typo, correction in typoLib.items():
            if typo in line:
                line = line.replace(typo, correction)

        # Autoreply and attachments are unlikely to have From: inside, but very likely to have To and Date.
        # If found them, turn back to only consider From as the beginning of an email
        # Only case to turn off appendingMode
        if "From:" in line:
            print("\nFrom: in line")
            spt = line.split("From:")
            if spt[0]:
                spt_arg.stack.append(" ** split from ** " + spt[0])
                completeEmail(email, sv_arg, spt_arg)
                email = defaultdict(list)
                sv_arg.line_nb = defaultdict(list)
                sv_arg.char_pos.beh["From:"] = sv_arg.char_pos_eol - len(spt[1])
                line = "From" + spt[1]

            if spt_arg.isAutoMessage or spt_arg.isAttachment:
                spt_arg.isAutoMessage = False
                spt_arg.isAttachment = False
                completeEmail(email, sv_arg, spt_arg)
                email = defaultdict(list)
                sv_arg.line_nb = defaultdict(list)

        elif "Subject:" in line:
            print("\nSubject: in line")
            spt = line.split("Subject:")
            if spt[0]:
                print(f"split subject, line: {line_no}")
                spt_arg.stack.append(" ** split Subject ** " + spt[0])
                print("GE, Subject: before completeEmail(0), line_no: ", line_no)
                completeEmail(email, sv_arg, spt_arg)
                email = defaultdict(list)
                sv_arg.line_nb = defaultdict(list)
                line = "Subject:" + spt[1]
                print(f"new subject line: {line}")
                
            if spt_arg.isAttachment:
                spt_arg.isAttachment = False
                completeEmail(email, sv_arg, spt_arg)
                email = defaultdict(list)
                sv_arg.line_nb = defaultdict(list)

        # append everything into body
        if spt_arg.isAutoMessage or spt_arg.isAttachment:
            # if by GE
            if spt_arg.isAutoMessage:
                spt_arg.stack.append(" ** autoMsg ** " + line)
            else:
                spt_arg.stack.append(" ** attach ** " + line)
            continue

        # if these string shows in line, it's probably the end of body. So everything below might be attachment.
        end_of_body = ["Tallahassee, FL", "Phone:", "FAX:"]
        for pattern in end_of_body:
            if pattern in line:
                spt_arg.isAttachment = True

        # Autoreplys or autoforwards are found. First of three cases triggers appendingMode.
        keywordsInAutomessage = [
            "Read:",
            "Not read:",
            "Sender:",
            "Tentative:",
            "Accepted:",
            "Declined:",
        ]
        for keyword in keywordsInAutomessage:
            if keyword in line:
                spt_arg.isAutoMessage = True
                email["isAutoMessage"] = True
                sv_arg.line_nb["isAutoMessage"].append(line_no)

        # Embedded attachments are found. Second of three cases triggers appendingMode.
        # Attachments usually have a title line that all capital letters
        # Add tolerance for all cap lines if comes right after a header section
        if (
            spt_arg.capTolerance <= 0
            and len(line) > 5
            and all(word.isupper() for word in line.split())
            and not "." in line
            and not ":" in line
        ):
            spt_arg.isAttachment = True
            spt_arg.stack.append(" ** attach1] ** " + line)
            email["hasAllCapLine"] = True
            sv_arg.line_nb["hasAllCapLine"].append(line_no)
            continue

        # if no pattern match, put into stack
        line_to_stack = True

        # search if line match any pattern
        for regex, section in headerReLib.items():
            if regex.match(line):
                # if any header section is found, the next line can be all cap without trigger appendingMode
                # capTolerance = 2 actually has 1 line tolerance. Because decrement is earlier than checking
                spt_arg.capTolerance = 2  # can be larger, but with cautious

                # found a repeating header section, indicating current email ends
                if section in email:
                    print(f"FOUND REPEATING SECTION: {section}")
                    # Bad date found. Third of three cases triggers appendingMode.
                    # This can be an attachment or an appointment
                    if section == "Sent" and not re_wrong_date.search(line):
                        spt_arg.stack.append(" ** stack, attachment ** ", line)
                        spt_arg.isAttachment = True
                        email["hasBadDate"] = True
                        sv_arg.line_nb["hasBadDate"].append(line_no)
                        line_to_stack = False
                        break
                    else:
                        print()
                        if section == "Subject":
                            print(
                                "GE, 2nd Subject: before completeEmail(1), line_no: ",
                                line_no,
                            )
                            print(f"  line[{line_no}: {line}")
                            # print(f"GE, line_no({prev_section}): {sv_arg.line_no['prev_section']}")
                            sv_arg.GE_subject_complete_email = True
                        print_email_linenb("-1-", email, sv_arg.line_nb)
                        print(f"Before call completeEmail (1)")
                        completeEmail(email, sv_arg, spt_arg)
                        print("GE: reset email and line_nb dictionaries\n")
                        email = defaultdict(list)
                        sv_arg.line_nb = defaultdict(list)
                        print_email_linenb("--1--", email, sv_arg.line_nb)
                        print("after --1--")
                        # the subject is now empty, but the key still exists. . Do not understand

                # section not in email, all lines in stack belongs to prev_section
                elif spt_arg.stack:
                    print()
                    print("stack")
                    if spt_arg.prev_section == None:
                        print(
                            "If this is not happenning at the beginning of a file, it is an error"
                        )
                        print("current line no:", line_no)
                        spt_arg.stack = []
                    else:
                        email[spt_arg.prev_section] = (
                            email[spt_arg.prev_section] + [f" ** prev_section1: {spt_arg.prev_section} ** "] + spt_arg.stack
                        )
                        sv_arg.line_nb[spt_arg.prev_section].append(line_no)
                        spt_arg.stack = []

                    if spt_arg.displacement_sections:
                        print(f"if spt_arg.displacement_sections, line_no: {line_no}")
                        print_email_linenb("-2-", email, sv_arg.line_nb)
                        pop = spt_arg.displacement_sections.pop(0)
                        print(f"({sp_arg.line_no}): pop displacement_sections: ", pop)

                # get rid of the header
                line = regex.findall(line)[0]
                print()
                
                # in Subject: above, the email was reset because the header appeared for a second time.
                # But the line remains in play. The header could become a part of a new email. 

                email[section].append(f" ** section {section} ** " + line)
                sv_arg.line_nb[section].append(line_no)  # last line nb
                print(
                    f"header: {section}, [{line_no}], content: {sv_arg.line_nb[section]}"
                )
                print(f"       appended: {line}")

                # found empty, maybe it's a block displacement
                print()
                if line == "":
                    print(f"line is empty: two quotes. line_no: {line_no}")
                    print_email_linenb("-3-", email, sv_arg.line_nb)
                    spt_arg.displacement_sections.append(" ** empty ** " + section)
                    print(
                        f"GE,     displacement_sections: ",
                        spt_arg.displacement_sections,
                    )
                    spt_arg.capTolerance += (
                        1  # if found empty header, there can have one more line all cap
                    )
                spt_arg.prev_section = section

                # if match, don't append to stack.
                # have to use flag here, because there are multiple patterns testing
                line_to_stack = False
                break

        if line_to_stack:
            spt_arg.stack.append(" ** line_to_stack ** " + line)

        # if block displacement is found, and there are lines in the stack
        print()
        if spt_arg.displacement_sections and spt_arg.stack:
            print(
                f"in displacement_sections and stack is NOT empty, line_no: {line_no}"
            )
            print(f"   stack: {spt_arg.stack}")
            print_email_linenb("before: ", email, sv_arg.line_nb)

            spt_arg.isDisplacement = True
            email["isDisplacement"] = True
            sv_arg.line_nb["isDisplacement"].append(line_no)
            popline = spt_arg.stack.pop(0)
            print(f"   displacement_section, popline: {popline}")
            section = spt_arg.displacement_sections.pop(0)
            print(f"   current section: {section}")
            if section == "Sent":
                is_date = fct_lib.is_date_valid(popline, filters)
                if is_date:
                    email["Sent"].append(" ** is_date ** " + popline)  # MIGHT NOT BE THE SOLUTION
                else:
                    # Either continue the line, or put this line with the previous header
                    email[spt_arg.prev_section].append(" ** sent, not isDate ** " + popline)
            else:
                email[section].append(" ** sent else ** " + popline)

            print_email_linenb("after: ", email, sv_arg.line_nb)

            sv_arg.line_nb[section].append(line_no)
            spt_arg.prev_section = section

    # don't forget the last email
    completeEmail(email, sv_arg, spt_arg)


- Initialize SavingArgs with an inputpath and an outputpath as 
```python
sv_arg = SavingArgs(inputpath = "./test_input/", outputpath = "./test_output/")
```
- For single file
```python
sv_arg.infile = "9-1-Adam-Corey-2012-1-0.txt"
process(sv_arg)
```
- For multiple files
```python
for file in filenames:
    sv_arg.infile = file
    process(sv_arg)
```

**Note**
If you want to rerun, it's better to reinitialize sv_arg, because sv_arg.email_count keeps increasing.

In [ ]:
sv_ge = SavingArgs(inputpath= "./email_txt/test_files/", outputpath = "./gordon4/", email_count=0)
sv_ge.infile = "gordon4.txt"
process(sv_ge)

In [ ]:
sv_ge = SavingArgs(inputpath= "./gordon5/", outputpath = "./gordon5/", email_count=0)
sv_ge.infile = "gordon5.txt"
process(sv_ge)

In [108]:
sv_ge = SavingArgs(inputpath= "./gordon6/", outputpath = "./gordon6/", email_count=0)
sv_ge.infile = "gordon6.txt"
process(sv_ge)

GE: Email not saved, line_no: 299, valid_count: 0
GE: Email not saved, line_no: 2006, valid_count: 1


In [15]:
sv_ge = SavingArgs(inputpath= "./gordon7/", outputpath = "./gordon7/", email_count=0)
sv_ge.infile = "gordon7.txt"
process(sv_ge)

In [16]:
sv_ge = SavingArgs(inputpath= "./23-11/", outputpath = "./23-11/", email_count=0)
sv_ge.infile = "23-11-PCSgmail2014-2017.txt"
process(sv_ge)

In [17]:
sv_ge = SavingArgs(inputpath= "./17-2/", outputpath = "./17-2/", email_count=0)
sv_ge.infile = "17-2_subset.txt"
process(sv_ge)

In [18]:
sv_ge = SavingArgs(inputpath= "./17-2a/", outputpath = "./17-2a/", email_count=0)
sv_ge.infile = "17-2.txt"
process(sv_ge)

In [19]:
sv_ge = SavingArgs(inputpath= "./17-2b/", outputpath = "./17-2b/", email_count=0)
sv_ge.infile = "17-2-IB2013-1-0.txt"
process(sv_ge)

In [20]:
sv_ge = SavingArgs(inputpath= "./23-11a/", outputpath = "./23-11a/", email_count=0)
sv_ge.infile = "23-11.txt"
process(sv_ge)

In [21]:
sv_ge = SavingArgs(inputpath= "./34-3/", outputpath = "./34-3/", email_count=0)
sv_ge.infile = "34-3.txt"
process(sv_ge)

In [22]:
sv_ge = SavingArgs(inputpath= "./18-4/", outputpath = "./18-4/", email_count=0)
sv_ge.infile = "18-4.txt"
process(sv_ge)

In [23]:
sv_ge = SavingArgs(inputpath= "./18-4/", outputpath = "./18-4/", email_count=0)
sv_ge.infile = "18-4a.txt"
process(sv_ge)

In [24]:
sv_ge = SavingArgs(inputpath= "./18-4/", outputpath = "./18-4/", email_count=0)
sv_ge.infile = "18-4b.txt"
process(sv_ge)

In [25]:
sv_ge = SavingArgs(inputpath= "./18-4/", outputpath = "./18-4/", email_count=0)
sv_ge.infile = "18-4d.txt"
process(sv_ge)

In [26]:
sv_ge = SavingArgs(inputpath= "./34-1/", outputpath = "./34-1/", email_count=0)
sv_ge.infile = "34-1a.txt"
process(sv_ge)

In [27]:
sv_ge = SavingArgs(inputpath= "./34-1/", outputpath = "./34-1/", email_count=0)
sv_ge.infile = "34-1b.txt"
process(sv_ge)

In [30]:
sv_ge = SavingArgs(inputpath= "./34-1/", outputpath = "./34-1/", email_count=0)
sv_ge.infile = "34-1c.txt"
process(sv_ge)

In [14]:
def process_case(path, filenm):
    sv_ge = SavingArgs(inputpath= path, outputpath = path, email_count=0)
    sv_ge.infile = filenm
    process(sv_ge)

In [39]:
process_case("./21-10/", "21-10.txt")

In [40]:
process_case("./21-11/", "21-11.txt")

In [41]:
process_case("./23-4/", "23-4.txt")

process_case("./21-6/", "21-6.txt")

In [190]:
# files = ["test1", "test2", "test3"] 
# files = ["test20", "test21"]
files = ["test22"]
#files = ["test23"]
for f in files:
    process_case("./10-8/", f + ".txt")


From: in line

header: From, [0], content: [0]
       appended: Google Calendar on behalf of Ryan Grindler <rgrindler@




stack

header: Sent, [3], content: [3]
       appended: Wednesday, May 18, 2016 10:35 AM



header: To, [5], content: [5]
       appended: Donahoe, Judy



Subject: in line

header: Subject, [7], content: [7]
       appended: Grass walkways at Edison



header: Attachments, [9], content: [9]
       appended: invite.ics




Subject: in line
FOUND REPEATING SECTION: Subject

GE, 2nd Subject: before completeEmail(1), line_no:  14
  line[14: Subject:
*** email, line_nb: -1- ***
header: From: [0, 3]
        [' ** section From ** Google Calendar on behalf of Ryan Grindler <rgrindler@', ' ** prev_section: From ** ', ' ** line_to_stack ** lOltally.com']
header: Sent: [3]
        [' ** section Sent ** Wednesday, May 18, 2016 10:35 AM']
header: To: [5]
        [' ** section To ** Donahoe, Judy']
header: Subject: [7]
        [' ** section Subject ** Grass walkways at Edison'

In [186]:
sv_ge = SavingArgs(inputpath= "./email_txt/downloaded_from_data_tallahassee/", outputpath = "./test_output/")


sv_ge.email_couant = 0   # IMPORTANT to reset email_count

files = os.listdir(sv_ge.inputpath)

for inputfile in tqdm(files[0:]):
    sv_ge.infile = inputfile
    # print(sv_ge.inputpath, inputfile)
    process(sv_ge)

  0%|          | 0/124 [00:00<?, ?it/s]


From: in line

header: From, [0], content: [0]
       appended: Bowen, Dan



header: Sent, [1], content: [1]
       appended: Tuesday, April 22, 2014 3:34 PM



header: To, [2], content: [2]
       appended: 'paigecartersmith@gmail.com'



header: CC, [3], content: [3]
       appended: 'electricalsrvs@aol.com'; Ash, Richard



Subject: in line

header: Subject, [4], content: [4]
       appended: 902 NORTH ADAMS ST



header: Attachments, [5], content: [5]
       appended: maddox building (5.37 KB)









From: in line
line_no: [0], key: From
line_no: [1], key: Sent
line_no: [2], key: To
line_no: [3], key: CC
line_no: [], key: Bcc
line_no: [4], key: Subject
line_no: [5], key: Attachments
line_no: [], key: Importance
line_no: [26], key: Body
line_no: [], key: isThread
line_no: [], key: isAutoMessage
line_no: [], key: isDisplacement
line_no: [14], key: hasAllCapLine
line_no: [], key: hasBadDate
GE: Email saved, line_no: 26, valid_count: 3
GE: Reset stack, displacement_sections, prev_s

TypeError: unsupported operand type(s) for -: 'collections.defaultdict' and 'int'

In [37]:
sv_ge = SavingArgs(inputpath= "./email_txt/downloaded_from_data_tallahassee/", outputpath = "./test_output/")


sv_ge.email_count = 0   # IMPORTANT to reset email_count

files = os.listdir(sv_ge.inputpath)

for inputfile in tqdm(files[0:]):
    sv_ge.infile = inputfile
    # print(sv_ge.inputpath, inputfile)
    process(sv_ge)

  0%|          | 0/124 [00:00<?, ?it/s]

If this is not happenning at the beginning of a file, is an error
current line no: 1
If this is not happenning at the beginning of a file, is an error
current line no: 2
If this is not happenning at the beginning of a file, is an error
current line no: 2
If this is not happenning at the beginning of a file, is an error
current line no: 1
If this is not happenning at the beginning of a file, is an error
current line no: 2
If this is not happenning at the beginning of a file, is an error
current line no: 1
If this is not happenning at the beginning of a file, is an error
current line no: 2
If this is not happenning at the beginning of a file, is an error
current line no: 2
If this is not happenning at the beginning of a file, is an error
current line no: 2
If this is not happenning at the beginning of a file, is an error
current line no: 2
If this is not happenning at the beginning of a file, is an error
current line no: 2
If this is not happenning at the beginning of a file, is an error

# non function code below